In [1]:
import nltk
import re
import pandas as pd
import numpy as np
import sklearn
from difflib import SequenceMatcher
from sklearn import model_selection

In [2]:
#Import Data
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
attributes=pd.read_csv('attributes.csv')

In [3]:
#Remove Nan
attributes=attributes.dropna(how='all')
attributes.reset_index(inplace=True)

In [4]:
#Get a stats of attributes names
all_names=attributes.groupby('name').count().sort_values('index',ascending=False)

In [5]:
attributes.product_uid.unique().__len__()

86263

In [6]:
#Attributes with al least 5% occurace
all_names.drop(all_names[all_names.value<4313].index)

,index,product_uid,value
name,,,
MFG Brand Name,86250,86250,86220
Bullet02,86248,86248,86248
Bullet03,86226,86226,86226
Bullet04,86174,86174,86174
Bullet01,85940,85940,85940
Product Width (in.),61137,61137,61137
Bullet05,60529,60529,60528
Product Height (in.),54698,54698,54698
Product Depth (in.),53652,53652,53652


In [7]:
attributes_2=attributes.copy()

In [73]:
def add_string(df):
    return reduce((lambda x, y: str(x)+' '+str(y)),df.tolist())
Color_Finish=attributes_2.loc[map(lambda x:re.search('(olor)|(inish)',x)!=None,attributes_2['name'])].groupby('product_uid').agg({'value':[add_string]})
Color_Finish.columns=['Color_Finish']
Bullets=attributes_2.loc[map(lambda x:re.search('(Bullet)',x)!=None,attributes_2['name'])].groupby('product_uid').agg({'value':[add_string]})
Bullets.columns=['Bullets']
Materials=attributes_2.loc[map(lambda x:re.search('(M|m)aterial',x)!=None,attributes_2['name'])].groupby('product_uid').agg({'value':[add_string]})
Materials.columns=['Materials']
Types=attributes_2.loc[map(lambda x:re.search('Product Type',x)!=None,attributes_2['name'])].groupby('product_uid').agg({'value':[add_string]})
Types.columns=['Types']

In [76]:
Attr_1=pd.merge(pd.merge(pd.merge(Color_finish,Bullets,left_index=True,right_index=True),Materials,left_index=True,right_index=True),Types,left_index=True,right_index=True)

In [82]:
Attr_2=pd.merge(Attr_1,test.groupby('product_uid').agg({'product_title':'first'}),left_index=True,right_index=True)

In [83]:
Attr_2

,"(value, add_string)",Bullets,Materials,Types,product_title
product_uid,,,,,
100012,Tan Caramel Browns/Tans Fruitwood,96 in. W x 72 in. L Actual Blind Width is 96 i...,Bamboo,Window Treatment Natural/Woven Roman Blind/Shade,Hampton Bay Caramel Simple Weave Bamboo Rollup...
100016,Blacks,Vented canopy Built in lattice work is great f...,Steel,Gazebo,Sunjoy Calais 8 ft. x 5 ft. x 8 ft. Steel Tile...
100032,Gray Gray,"Versatile enough for the garage, basement, lau...",PVC,Side and Wall Panels,NorskWall Slatwall Panel Kit (4-Piece)
100036,White White,No installation tools required; bracket attach...,Metal,Brackets,Rubbermaid 12 in. D Single Track Bracket
100037,Black Black,3 main pockets and 6 tool pockets 4 interior e...,Fabric,Pouch Hand Tool,Husky 9-Pocket Maintenance Pouch
100040,White White,"Corresponding posts sold separately, use: Line...",Vinyl,Vinyl Fence Panels,Veranda Washington 4 ft. x 6 ft. White Vinyl U...
100045,Oil Rubbed Bronze Bronze,"2-1/2 in. center to center measurement, 4-13/3...",Metal,Cabinet Pull,Sumner Street Home Hardware Grayson 2-1/2 in. ...
100046,Green Green,Manufactured from rail steel Weighs 1.25 lbs. ...,Metal,Ornamental Posts,HDX 6 ft. Heavy Duty Steel Green Painted T-Post
100050,Unfinished Bare Brown,Part of the Create Your Own collection program...,Foam Steel,Patio Furniture Seating Set,Hampton Bay Beverly Patio Dining Arm Chairs wi...


In [85]:
Attr_2.shape

(17191, 5)

In [87]:
Attr_1.shape

(20680, 4)

In [89]:
Attr_2.columns=['Color_Finish','Bullets','Materials','Types','Title']

In [90]:
Attr_2

,Color_Finish,Bullets,Materials,Types,Title
product_uid,,,,,
100012,Tan Caramel Browns/Tans Fruitwood,96 in. W x 72 in. L Actual Blind Width is 96 i...,Bamboo,Window Treatment Natural/Woven Roman Blind/Shade,Hampton Bay Caramel Simple Weave Bamboo Rollup...
100016,Blacks,Vented canopy Built in lattice work is great f...,Steel,Gazebo,Sunjoy Calais 8 ft. x 5 ft. x 8 ft. Steel Tile...
100032,Gray Gray,"Versatile enough for the garage, basement, lau...",PVC,Side and Wall Panels,NorskWall Slatwall Panel Kit (4-Piece)
100036,White White,No installation tools required; bracket attach...,Metal,Brackets,Rubbermaid 12 in. D Single Track Bracket
100037,Black Black,3 main pockets and 6 tool pockets 4 interior e...,Fabric,Pouch Hand Tool,Husky 9-Pocket Maintenance Pouch
100040,White White,"Corresponding posts sold separately, use: Line...",Vinyl,Vinyl Fence Panels,Veranda Washington 4 ft. x 6 ft. White Vinyl U...
100045,Oil Rubbed Bronze Bronze,"2-1/2 in. center to center measurement, 4-13/3...",Metal,Cabinet Pull,Sumner Street Home Hardware Grayson 2-1/2 in. ...
100046,Green Green,Manufactured from rail steel Weighs 1.25 lbs. ...,Metal,Ornamental Posts,HDX 6 ft. Heavy Duty Steel Green Painted T-Post
100050,Unfinished Bare Brown,Part of the Create Your Own collection program...,Foam Steel,Patio Furniture Seating Set,Hampton Bay Beverly Patio Dining Arm Chairs wi...
